In [3]:
import os
from aco import ACO
import sys
import numpy as np
from scipy.spatial import distance_matrix
import logging
import inspect


N_ANTS = 30
N_ITERATIONS = [1, 10, 30, 50, 100, 150, 200]
CAPACITY = 50

def heuristics_reevo(distance_matrix: np.ndarray, coordinates: np.ndarray, demands: np.ndarray, capacity: int) -> np.ndarray:
    num_nodes = distance_matrix.shape[0]
    
    # Calculate the inverse of the distance matrix
    inverse_distance_matrix = np.divide(1, distance_matrix, where=(distance_matrix != 0))
    
    # Calculate total demand and average demand
    total_demand = np.sum(demands)
    average_demand = total_demand / num_nodes
    
    # Calculate the distance from each node to the starting depot
    depot_distances = distance_matrix[:, 0]
    
    # Calculate the remaining capacity of the vehicle for each node
    remaining_capacity = capacity - demands
    
    # Initialize the heuristic matrix
    heuristic_matrix = np.zeros_like(distance_matrix)
    
    # Calculate the demand factor and distance factor
    demand_factor = demands / total_demand
    normalized_distance = distance_matrix / np.max(distance_matrix)
    distance_factor = depot_distances / (normalized_distance + np.finfo(float).eps)
    
    # Iterate over each node
    for i in range(num_nodes):
        
        # Calculate the heuristic value based on distance and capacity constraints
        heuristic_values = inverse_distance_matrix[i] * (1 / (normalized_distance[i] ** 2))
        
        # Adjust the heuristic values based on the remaining capacity
        heuristic_values = np.where(remaining_capacity >= demands[i], heuristic_values, 0)
        
        # Adjust the heuristic values based on the demand factor
        heuristic_values *= demand_factor[i] / average_demand
        
        # Adjust the heuristic values based on the distance factor
        heuristic_values *= distance_factor[i]
        heuristic_values[0] = 0  # Exclude the depot node
        
        # Adjust the heuristic values based on the capacity utilization
        utilization_factor = np.where(remaining_capacity >= demands[i], capacity - demands[i], 0)
        heuristic_values *= utilization_factor
        
        # Set the heuristic values for the current node in the heuristic matrix
        heuristic_matrix[i] = heuristic_values
    
    return heuristic_matrix

def solve(node_pos, demand):
    dist_mat = distance_matrix(node_pos, node_pos)
    dist_mat[np.diag_indices_from(dist_mat)] = 1 # set diagonal to a large number
    heu = heuristics_reevo(dist_mat, node_pos, demand, CAPACITY) + 1e-9
    heu[heu < 1e-9] = 1e-9
    aco = ACO(dist_mat, demand, heu, CAPACITY, n_ants=N_ANTS)
    
    results = []
    for i in range(len(N_ITERATIONS)):
        if i == 0:
            obj = aco.run(N_ITERATIONS[i])
        else:
            obj = aco.run(N_ITERATIONS[i] - N_ITERATIONS[i-1])
        # print("Iteration: {}, Objective: {}".format(N_ITERATIONS[i], obj))
        results.append(obj.item())
    return results



In [4]:
print("[*] Running ...")
    

for problem_size in [20, 50, 100]:
    dataset_path = f"dataset/val{problem_size}_dataset.npy"
    dataset = np.load(dataset_path)
    demands, node_positions = dataset[:, :, 0], dataset[:, :, 1:]
    
    n_instances = node_positions.shape[0]
    logging.info(f"[*] Evaluating {dataset_path}")
    
    objs = []
    for i, (node_pos, demand) in enumerate(zip(node_positions, demands)):
        obj = solve(node_pos, demand)
        objs.append(obj)
    
    # Average objective value for all instances
    mean_obj = np.mean(objs, axis=0)
    for i, obj in enumerate(mean_obj):
        print(f"[*] Average for {problem_size}, {N_ITERATIONS[i]} iterations: {obj}")
    print()

[*] Running ...
[*] Average for 20, 1 iterations: 5.359106900150456
[*] Average for 20, 10 iterations: 4.856290960054639
[*] Average for 20, 30 iterations: 4.780728418281577
[*] Average for 20, 50 iterations: 4.763402845346687
[*] Average for 20, 100 iterations: 4.739372008116089
[*] Average for 20, 150 iterations: 4.729715278567657
[*] Average for 20, 200 iterations: 4.725332738131605

[*] Average for 50, 1 iterations: 10.742579572004573
[*] Average for 50, 10 iterations: 9.711407913619976
[*] Average for 50, 30 iterations: 9.265205121184207
[*] Average for 50, 50 iterations: 9.11182004021968
[*] Average for 50, 100 iterations: 8.988891687862166
[*] Average for 50, 150 iterations: 8.926836021977824
[*] Average for 50, 200 iterations: 8.906123444482922

[*] Average for 100, 1 iterations: 18.326012464706558
[*] Average for 100, 10 iterations: 16.93568059110451
[*] Average for 100, 30 iterations: 16.08912256400286
[*] Average for 100, 50 iterations: 15.766953335921835
[*] Average for 100